In [1]:
# !npx esbuild ../js/widget.jsx --minify --format=esm --bundle --outdir=../static

In [2]:
import sys

sys.path.insert(0, "/Users/joerg/git_libs/pyiron_core")
# sys.path

In [3]:
from pyiron_workflow import Workflow, as_out_dataclass_node, as_function_node
from pyironflow.wf_extensions import get_nodes, _to_jsonifyable
import pyiron_nodes as pn
import numpy as np
import json
import pyiron_workflow.wf_graph_tools as gt

In [4]:
lammps = pn.atomistic.engine.lammps.InitLammps(label="lammps")
lammps.inputs

,label,type,default,ready,value,node
0,structure,NonPrimitive,NotData,False,NotData,<pyiron_workflow.simple_workflow.Node object a...
1,potential,str,NotData,False,NotData,<pyiron_workflow.simple_workflow.Node object a...
2,calculator,None,NotData,False,NotData,<pyiron_workflow.simple_workflow.Node object a...
3,working_directory,str,NotData,False,NotData,<pyiron_workflow.simple_workflow.Node object a...
4,create_dir,bool,True,True,True,<pyiron_workflow.simple_workflow.Node object a...


In [21]:
from pyiron_workflow.graph import base
from pyironflow.reactflow import ReactFlowWidget

sub_graph = base.Graph(label="subgraph")
sub_graph += pn.atomistic.structure.build.Bulk(name='Al', label="structure2")
sub_graph += pn.atomistic.structure.transform.Repeat(label="repeat2")
sub_graph += base.GraphEdge('structure2', 'repeat2', 'structure', 'structure')

# graph = base.Graph() # base.create_graph()
graph = base.Graph()
graph += pn.atomistic.structure.build.Bulk(name="Al", label="structure")
graph += pn.atomistic.structure.transform.Repeat(label="repeat")
# graph += sub_graph
graph += pn.atomistic.engine.lammps.Code(label="lammps") 
graph += base.GraphEdge("structure", "repeat", "structure", "structure")
graph += base.GraphEdge("repeat", "lammps", "structure", "structure")

# base._mark_node_as_collapsed(graph, 'lammps')
base.GuiGraph(graph)


structure:  NotData
structure:  NotData


ReactFlowWidget()

In [53]:
# base._nodes_to_gui(graph)
graph.nodes

,id,import_path,label,parent_id,level,node,graph,node_type,widget_type,expanded
0,structure,pyiron_nodes.atomistic.structure.build.Bulk,structure,None,0,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
1,repeat,pyiron_nodes.atomistic.structure.transform.Repeat,repeat,None,0,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
2,lammps,None,lammps,None,0,<pyiron_workflow.simple_workflow.Node object a...,"Graph(id='lammps', label='lammps', root_node=<...",graph,customNode,True
3,var_lammps__structure,pyiron_workflow.graph.base.identity,var_lammps__structure,lammps,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
4,var_lammps__potential,pyiron_workflow.graph.base.identity,var_lammps__potential,lammps,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
5,var_lammps__calculator,pyiron_workflow.graph.base.identity,var_lammps__calculator,lammps,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
6,var_lammps__working_directory,pyiron_workflow.graph.base.identity,var_lammps__working_directory,lammps,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
7,var_lammps__calc_mode,pyiron_workflow.graph.base.identity,var_lammps__calc_mode,lammps,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
8,InitLammps,pyiron_nodes.atomistic.engine.lammps.InitLammps,InitLammps,lammps,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
9,Shell,pyiron_nodes.atomistic.engine.lammps.Shell,Shell,lammps,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False


In [59]:
def get_inner_edge(graph, edge):
    inner_source = f'var_{edge.target}__{edge.targetHandle}'
    df = base._edges_to_gui(graph).df
    matching_edge = df.loc[df['source'] == inner_source].iloc[0].to_dict()
    new_edge = base.GraphEdge(edge.source, matching_edge["target"], edge.sourceHandle, matching_edge["targetHandle"]) 
    return new_edge
    

get_inner_edge(graph, graph.edges[11])


GraphEdge(source='repeat', target='InitLammps', sourceHandle='structure', targetHandle='structure')

In [54]:
graph.edges[11]

GraphEdge(source='repeat', target='lammps', sourceHandle='structure', targetHandle='structure')

In [8]:
graph.nodes

,id,import_path,label,parent_id,level,node,graph,node_type,widget_type,expanded
0,structure,pyiron_nodes.atomistic.structure.build.Bulk,structure,None,0,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
1,repeat,pyiron_nodes.atomistic.structure.transform.Repeat,repeat,None,0,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
2,lammps,None,lammps,None,0,<pyiron_workflow.simple_workflow.Node object a...,"Graph(id='lammps', label='lammps', root_node=<...",graph,customNode,True
3,var_lammps__structure,pyiron_workflow.graph.base.identity,var_lammps__structure,lammps,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
4,var_lammps__potential,pyiron_workflow.graph.base.identity,var_lammps__potential,lammps,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
5,var_lammps__calculator,pyiron_workflow.graph.base.identity,var_lammps__calculator,lammps,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
6,var_lammps__working_directory,pyiron_workflow.graph.base.identity,var_lammps__working_directory,lammps,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
7,var_lammps__calc_mode,pyiron_workflow.graph.base.identity,var_lammps__calc_mode,lammps,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
8,InitLammps,pyiron_nodes.atomistic.engine.lammps.InitLammps,InitLammps,lammps,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
9,Shell,pyiron_nodes.atomistic.engine.lammps.Shell,Shell,lammps,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False


In [9]:
base.get_graph_from_macro(graph.nodes['lammps'].node).label

structure:  NotData


In [10]:
base._get_active_nodes(graph)

,id,import_path,label,parent_id,level,node,graph,node_type,widget_type,expanded
0,structure,pyiron_nodes.atomistic.structure.build.Bulk,structure,None,0,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
1,repeat,pyiron_nodes.atomistic.structure.transform.Repeat,repeat,None,0,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
2,lammps,None,lammps,None,0,<pyiron_workflow.simple_workflow.Node object a...,"Graph(id='lammps', label='lammps', root_node=<...",graph,customNode,True
3,var_lammps__structure,pyiron_workflow.graph.base.identity,var_lammps__structure,lammps,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
4,var_lammps__potential,pyiron_workflow.graph.base.identity,var_lammps__potential,lammps,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
5,var_lammps__calculator,pyiron_workflow.graph.base.identity,var_lammps__calculator,lammps,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
6,var_lammps__working_directory,pyiron_workflow.graph.base.identity,var_lammps__working_directory,lammps,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
7,var_lammps__calc_mode,pyiron_workflow.graph.base.identity,var_lammps__calc_mode,lammps,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
8,InitLammps,pyiron_nodes.atomistic.engine.lammps.InitLammps,InitLammps,lammps,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
9,Shell,pyiron_nodes.atomistic.engine.lammps.Shell,Shell,lammps,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False


In [11]:
base._find_input_nodes(graph, 2)

[0, 1, 2]

In [12]:
base.GuiGraph(base.get_graph_from_macro(graph.nodes['lammps'].node))

structure:  NotData


ReactFlowWidget()

In [13]:
graph.nodes

,id,import_path,label,parent_id,level,node,graph,node_type,widget_type,expanded
0,structure,pyiron_nodes.atomistic.structure.build.Bulk,structure,None,0,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
1,repeat,pyiron_nodes.atomistic.structure.transform.Repeat,repeat,None,0,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
2,lammps,None,lammps,None,0,<pyiron_workflow.simple_workflow.Node object a...,"Graph(id='lammps', label='lammps', root_node=<...",graph,customNode,True
3,var_lammps__structure,pyiron_workflow.graph.base.identity,var_lammps__structure,lammps,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
4,var_lammps__potential,pyiron_workflow.graph.base.identity,var_lammps__potential,lammps,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
5,var_lammps__calculator,pyiron_workflow.graph.base.identity,var_lammps__calculator,lammps,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
6,var_lammps__working_directory,pyiron_workflow.graph.base.identity,var_lammps__working_directory,lammps,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
7,var_lammps__calc_mode,pyiron_workflow.graph.base.identity,var_lammps__calc_mode,lammps,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
8,InitLammps,pyiron_nodes.atomistic.engine.lammps.InitLammps,InitLammps,lammps,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False
9,Shell,pyiron_nodes.atomistic.engine.lammps.Shell,Shell,lammps,1,<pyiron_workflow.simple_workflow.Node object a...,None,node,customNode,False


In [14]:
node_to_index = {node_label: index for index, node_label in enumerate(graph.nodes.keys()) }
node_to_index

{'structure': 0,
 'repeat': 1,
 'lammps': 2,
 'var_lammps__structure': 3,
 'var_lammps__potential': 4,
 'var_lammps__calculator': 5,
 'var_lammps__working_directory': 6,
 'var_lammps__calc_mode': 7,
 'InitLammps': 8,
 'Shell': 9,
 'ParseLogFile': 10,
 'ParseDumpFile': 11,
 'Collect': 12}

In [15]:
base.display_gui_data(graph)

,label,source_labels,target_labels,import_path,target_values,target_types,source_values,source_types
0,structure,[structure],"[name, crystalstructure, a, c, c_over_a, u, or...",pyiron_nodes.atomistic.structure.build.Bulk,"[Al, None, None, None, None, None, False, False]","[str, str, float, float, int, float, bool, bool]",[NotData],[builtins.NoneType]
1,repeat,[structure],"[structure, repeat_scalar]",pyiron_nodes.atomistic.structure.transform.Repeat,"[NotData, 1]","[NonPrimitive, int]",[NotData],[builtins.str]
2,lammps,[],[],None,[],[],[],[]
3,var_lammps__structure,[x],[x],pyiron_workflow.graph.base.identity,[NotData],[None],[NotData],[builtins.NoneType]
4,var_lammps__potential,[x],[x],pyiron_workflow.graph.base.identity,[NotData],[None],[NotData],[builtins.NoneType]
5,var_lammps__calculator,[x],[x],pyiron_workflow.graph.base.identity,[NotData],[None],[NotData],[builtins.NoneType]
6,var_lammps__working_directory,[x],[x],pyiron_workflow.graph.base.identity,[NotData],[None],[NotData],[builtins.NoneType]
7,var_lammps__calc_mode,[x],[x],pyiron_workflow.graph.base.identity,[NotData],[None],[NotData],[builtins.NoneType]
8,InitLammps,[path],"[structure, potential, calculator, working_dir...",pyiron_nodes.atomistic.engine.lammps.InitLammps,"[NotData, None, NotData, test2, True]","[NonPrimitive, str, None, str, bool]",[NotData],[builtins.NoneType]
9,Shell,"[output, dump, log]","[working_directory, command, environment, argu...",pyiron_nodes.atomistic.engine.lammps.Shell,"[NotData, lmp, None, None]","[None, str, NonPrimitive, NonPrimitive]","[NotData, NotData, NotData]","[builtins.NoneType, builtins.NoneType, builtin..."


In [16]:
base._edges_to_gui(graph)

,source,target,sourceHandle,targetHandle,id,style
0,InitLammps,Shell,path,working_directory,0,"{'strokeWidth': 2, 'stroke': 'black'}"
1,Shell,ParseLogFile,log,log_file,1,"{'strokeWidth': 2, 'stroke': 'black'}"
2,Shell,ParseDumpFile,dump,dump_file,2,"{'strokeWidth': 2, 'stroke': 'black'}"
3,ParseDumpFile,Collect,dump,out_dump,3,"{'strokeWidth': 2, 'stroke': 'black'}"
4,ParseLogFile,Collect,log,out_log,4,"{'strokeWidth': 2, 'stroke': 'black'}"
5,var_lammps__structure,InitLammps,x,structure,5,"{'strokeWidth': 2, 'stroke': 'black'}"
6,var_lammps__potential,InitLammps,x,potential,6,"{'strokeWidth': 2, 'stroke': 'black'}"
7,var_lammps__calculator,InitLammps,x,calculator,7,"{'strokeWidth': 2, 'stroke': 'black'}"
8,var_lammps__working_directory,InitLammps,x,working_directory,8,"{'strokeWidth': 2, 'stroke': 'black'}"
9,var_lammps__calc_mode,Collect,x,calc_mode,9,"{'strokeWidth': 2, 'stroke': 'black'}"


In [17]:
from pyiron_workflow import Workflow

wf = Workflow("LammpsMacro")

wf.structure = pn.atomistic.structure.build.Bulk(name="Al")
wf.potential = pn.atomistic.engine.lammps.Potential()
wf.calculator = pn.atomistic.engine.lammps.CalcMD()

wf.InitLammps = pn.atomistic.engine.lammps.InitLammps(
    structure=wf.structure,
    potential=wf.potential, 
    calculator=wf.calculator,
    working_directory='test',
)

wf.Shell = pn.atomistic.engine.lammps.Shell(
    # command=ExecutablePathResolver(module="lammps", code="lammps").path(),
    working_directory=wf.InitLammps,
)

wf.ParseLogFile = pn.atomistic.engine.lammps.ParseLogFile(log_file=wf.Shell.outputs.log)
wf.ParseDumpFile = pn.atomistic.engine.lammps.ParseDumpFile(dump_file=wf.Shell.outputs.dump)
wf.Collect = pn.atomistic.engine.lammps.Collect(
    out_dump=wf.ParseDumpFile.outputs.dump,
    out_log=wf.ParseLogFile.outputs.log,
    calc_mode="md",  # wf.calc,
)

base.get_nodes_from_wf(wf)

AttributeError: module 'pyiron_workflow.graph.base' has no attribute 'get_nodes_from_wf'

In [ ]:
wf._nodes

In [ ]:
graph.nodes

In [ ]:
xx

### Test wf_graph.py

In [ ]:
from pyiron_workflow.wf_graph import Graph, GraphNode, GraphEdge, GuiData

In [ ]:
lammps = pn.atomistic.engine.lammps.Code()
macro_graph = Graph().get_graph_from_macro(lammps)

# w = macro_graph.display()
# w

In [ ]:
sub_graph = Graph(label="lammps")
sub_graph.add_node( pn.atomistic.structure.build.Bulk(name='Al'), label="structure2")
sub_graph.add_node( pn.atomistic.structure.transform.Repeat(), label="repeat2")
sub_graph.add_edge('structure2', 'repeat2', 'structure', 'structure')

# TODO: copy the graph to avoid changing the original graph


graph = Graph()
graph.add_node(pn.atomistic.structure.build.Bulk(name="Al"), label="structure")
graph.add_node(pn.atomistic.structure.transform.Repeat(), label="repeat")
graph.add_node(macro_graph, label="lammps")
graph.add_node(sub_graph, label="subgraph")
graph.add_edge("structure", "repeat", "structure", "structure")

# TODO: validate consistency of edges!!
if True:
    graph.add_edge("repeat", "InitLammps", "structure", "structure")
else:
    graph._mark_node_as_collapsed('lammps')
    graph._mark_node_as_collapsed('subgraph')
    graph.add_edge("repeat", "lammps", "structure", "structure")
    

w = graph.display()

w


Strategy:
- include all edges (to collapsed and expanded elements)
- add to active ones only the appropriate ones

Tasks:
- translate edge to macro node input into edge to node in macro input

In [ ]:
graph.nodes['lammps'].node.inputs

get non-connected input and output ports of all nodes inside the macro

In [ ]:
from pyiron_workflow.wf_graph_tools import get_graph_from_wf

get_graph_from_wf(lammps._wf_macro).nodes # List

In [ ]:
get_graph_from_wf(lammps._wf_macro).edges # List

In [ ]:
macro_graph.nodes # OrderedDict

In [ ]:
macro_graph.edges # OrderedDict of GraphEdge type objects! -> small but subtile differences

In [ ]:
macro_graph.display_nodes()

macro -> workflow transformation incomplete:
- non-connected input is missing
- input ports are not included
- ports should be converted into virtual nodes, which are connected to their respective nodes


In [ ]:
import inspect

def print_function_source(func):
    source_code = inspect.getsource(func)
    print(source_code)

# Example usage
print_function_source(graph.display)

In [ ]:
from pyiron_workflow import Workflow, as_function_node, as_macro_node, Node
from pyiron_nodes.atomistic.engine.lammps import Atoms, Optional, InitLammps, Shell, ParseLogFile, ParseDumpFile, Collect
from pyiron_workflow.wf_graph_tools import get_graph_from_wf

@as_function_node("generic")
def Code(
    structure: Atoms,
    calculator, # =InputCalcStatic(),  # TODO: Don't use mutable defaults
    potential: Optional[str] = None,
    working_dir: str = "test2",
):

    from pyiron_workflow import Workflow

    print("structure: ", structure)

    wf = Workflow("LammpsMacro")

    # wf.Potential = Potential(structure=structure, name=potential)

    # wf.ListPotentials = ListPotentials(structure=structure)

   #  wf.calc = Calc(calculator)

    wf.InitLammps = InitLammps(
        structure=structure,
        potential=potential, # wf.Potential,
        calculator=calculator,
        working_directory=working_dir,
    )

    wf.Shell = Shell(
        # command=ExecutablePathResolver(module="lammps", code="lammps").path(),
        working_directory=wf.InitLammps,
    )

    wf.ParseLogFile = ParseLogFile(log_file=wf.Shell.outputs.log)
    wf.ParseDumpFile = ParseDumpFile(dump_file=wf.Shell.outputs.dump)
    wf.Collect = Collect(
        out_dump=wf.ParseDumpFile.outputs.dump,
        out_log=wf.ParseLogFile.outputs.log,
        calc_mode="md",  # wf.calc,
    )

    return wf.Collect

@as_function_node("lammps")
def identity(x):
    return x




#inp_port_structure = Node(label="inp_port_structure", )
out = Code()._run()

wft = out._workflow

In [ ]:
lammps = pn.atomistic.engine.lammps.Code(label="lammps") 
lammps
# base.get_graph_from_macro(lammps)

In [ ]:
lammps.

In [ ]:
base.get_graph_from_wf(wft).edges

In [ ]:
# could be easily extended to go over all open ports of the subnodes
kwargs = {}
for inp in Code().inputs.data['label']:
    inp_port_label = f"inp_port_{inp}"
    kwargs[inp] = identity(inp_port_label, label=inp_port_label)

out = Code(**kwargs)._run()
base.get_graph_from_wf(out._workflow).edges

In [ ]:
wf_graph = base.get_graph_from_wf(out._workflow)
base.GuiGraph(wf_graph)

In [ ]:
out._workflow._nodes

In [ ]:
out._workflow._edges

In [ ]:
# transfer macro to subnode ports

In [ ]:
get_graph_from_wf(wft).nodes  # straightforward approach to get the missing edges between input port and subnode inputs!
# we only lack the sub node inputs that have no default parameters, this is ok (would be easy to get them)

In [ ]:
wft.add_node(label="inp_structure", node=inp_structure)


# wft.add_edge("inp_structure", "InitLammps", "structure", "structure")

In [ ]:
wft.child_labels

In [ ]:
wft._nodes

In [ ]:
out._workflow._edges

In [ ]:
graph.nodes['lammps'].graph.nodes["InitLammps"].node.inputs

In [ ]:
for key, node in graph.nodes['lammps'].graph.nodes.items():
    print(key, node.node.inputs)

In [ ]:
graph.nodes['lammps'].graph.nodes.keys()

In [ ]:

graph.display_nodes()

In [ ]:
graph.display_active_nodes()

In [ ]:
graph.display_gui_data()

In [ ]:
graph.display_edges()

In [ ]:
graph.display_active_edges()

In [ ]:
graph._edges_to_gui()